In [1]:
import json
import random
import re
from collections import Counter
from pathlib import Path

from participant import AnalyseLLMParticipant

import pandas as pd

/Users/njf/code/frank-llm/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
outputs = Path('outputs/select/zero').iterdir()
output = sorted(outputs)[-1]
with output.open() as f:
    data = json.load(f)

config = data['config']
for k, v in config.items():
    print(f'{k}: {v}')

T = AnalyseLLMParticipant(output)
T.df = T.df[T.df['template'] == 'C1']

MODEL: microsoft/Phi-3-small-128k-instruct
TEMPERATURE: 0.7
BATCH_SIZE: 20
SYSTEM_CONTENT: Your role is to select, from a list of steps, those that are most important for inclusion in a summary explanation of that process. Format your selection as a list. Use the following example to understand your task: [7, 3, 1, 9, 4]. Only output your selection in the form of a list and nothing else.
EXAMPLE: [7, 3, 1, 9, 4]
SHOT: 0
INPUT: resources/data/select.json
DESCRIPTION: Phi-3-small-128k-instruct:shot=0:temp=0.7
RUN_ID: c194c9d115724600aa7b68da9ed848b7
FORMAT_VERSION: 2


In [16]:
T.df

,qid,template,num_steps,system,user,generation,matches_format,valid_generation,response_eq_example,response_length_gt_steps,response_eq_all_steps,proportion_of_all_steps,meta_selections,meta_selections_pc,object_selections,object_selections_pc
300,8f58e5b8e08148c6,C1,10,None,"\nYour role is to select, from a list of steps...",. Question: What will be the life expectancy i...,"[1, 6, 7, 8, 9]","[1, 6, 7, 8, 9]",False,False,False,0.5,[1],0.333333,[],0.0
301,06912b44ca4349f3,C1,10,None,"\nYour role is to select, from a list of steps...",". Question: In 2030, what will be the populati...","[1, 3, 5, 7, 8, 9, 10]","[1, 3, 5, 7, 8, 9, 10]",False,False,False,0.7,"[1, 3, 5]",1.000000,[],0.0
302,81f657eccd4940f2,C1,10,None,"\nYour role is to select, from a list of steps...",. Question: What will be the energy consumptio...,"[1, 6, 7, 8, 9]","[1, 6, 7, 8, 9]",False,False,False,0.5,[1],0.333333,[],0.0
303,885da3aa1c124fa6,C1,10,None,"\nYour role is to select, from a list of steps...",". Question: In 2026, what will be the energy c...","[1, 3, 5, 6, 7, 9]","[1, 3, 5, 6, 7, 9]",False,False,False,0.6,"[1, 3, 5]",1.000000,[],0.0
304,bf1a680acd224df3,C1,10,None,"\nYour role is to select, from a list of steps...",. Question: What will be the rural population ...,"[1, 3, 6, 7, 8, 9]","[1, 3, 6, 7, 8, 9]",False,False,False,0.6,"[1, 3]",0.666667,[],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,f4b62e05b8bc4f1e,C1,10,None,"\nYour role is to select, from a list of steps...",. Question: What will be the birth rate in 202...,"[1, 3, 4, 6, 7, 9]","[1, 3, 4, 6, 7, 9]",False,False,False,0.6,"[1, 3]",0.666667,[4],0.5
396,c74a7f331d1b47bb,C1,10,None,"\nYour role is to select, from a list of steps...",. Question: For the country in Eastern Europe ...,"[1, 6, 7, 8, 9]","[1, 6, 7, 8, 9]",False,False,False,0.5,[1],0.333333,[],0.0
397,3d0b6bb28d244c29,C1,10,None,"\nYour role is to select, from a list of steps...",. Question: For the country in Northern Africa...,"[1, 4, 6, 7, 9]","[1, 4, 6, 7, 9]",False,False,False,0.5,[1],0.333333,[4],0.5
398,91c9d3a53be745f9,C1,10,None,"\nYour role is to select, from a list of steps...",". Question: In 2028, what will be the life exp...","[1, 4, 6, 8, 9]","[1, 4, 6, 8, 9]",False,False,False,0.5,[1],0.333333,[4],0.5


In [17]:
T.df['matches_format'].value_counts()

[1, 6, 7, 9]                 21
[1, 6, 7, 8, 9]              14
[1, 3, 5, 6, 7, 9]            8
[1, 3, 5, 6, 9]               5
[1, 3, 6, 7, 9]               4
[1, 3, 5, 7, 9]               4
[1, 3, 4, 5, 6, 9]            3
[1, 3, 4, 6, 7, 9]            3
[1, 3, 5, 6, 7, 8, 9, 10]     2
[1, 3, 5, 7, 8, 9]            2
[1, 3, 4, 5, 7, 8, 9, 10]     2
[1, 2, 4, 5, 6, 7, 9]         2
[1, 3, 6, 7, 8, 9]            2
[1, 3, 5, 7, 8, 9, 10]        2
[1, 4, 6, 7, 8, 9]            2
[1, 3, 5, 6, 8, 9, 10]        2
[1, 6, 7, 8, 9, 10]           2
[1, 3, 4, 6, 7, 9, 10]        2
[1, 3, 6, 7, 8, 9, 10]        2
[1, 3, 5, 7]                  1
[1, 2, 6, 7, 9]               1
[1, 3, 4, 5, 6, 8, 9]         1
[1, 3, 5, 6, 8, 9]            1
[1, 3, 4, 6, 8, 9]            1
[1, 4, 6, 7, 9]               1
[1, 4, 5, 6, 7, 9]            1
[1, 3, 6, 9]                  1
[1, 3, 6, 7, 9, 10]           1
[1, 3, 5, 6, 7, 8, 9]         1
[1, 3, 4, 5, 8, 9]            1
[1, 6, 9, 10]                 1
[1, 3, 4

In [18]:
T.df['valid_generation'].value_counts()

[1, 6, 7, 9]                 21
[1, 6, 7, 8, 9]              14
[1, 3, 5, 6, 7, 9]            8
[1, 3, 5, 6, 9]               5
[1, 3, 6, 7, 9]               4
[1, 3, 5, 7, 9]               4
[1, 3, 4, 5, 6, 9]            3
[1, 3, 4, 6, 7, 9]            3
[1, 3, 5, 6, 7, 8, 9, 10]     2
[1, 3, 5, 7, 8, 9]            2
[1, 3, 4, 5, 7, 8, 9, 10]     2
[1, 2, 4, 5, 6, 7, 9]         2
[1, 3, 6, 7, 8, 9]            2
[1, 3, 5, 7, 8, 9, 10]        2
[1, 4, 6, 7, 8, 9]            2
[1, 3, 5, 6, 8, 9, 10]        2
[1, 6, 7, 8, 9, 10]           2
[1, 3, 4, 6, 7, 9, 10]        2
[1, 3, 6, 7, 8, 9, 10]        2
[1, 3, 5, 7]                  1
[1, 2, 6, 7, 9]               1
[1, 3, 4, 5, 6, 8, 9]         1
[1, 3, 5, 6, 8, 9]            1
[1, 3, 4, 6, 8, 9]            1
[1, 4, 6, 7, 9]               1
[1, 4, 5, 6, 7, 9]            1
[1, 3, 6, 9]                  1
[1, 3, 6, 7, 9, 10]           1
[1, 3, 5, 6, 7, 8, 9]         1
[1, 3, 4, 5, 8, 9]            1
[1, 6, 9, 10]                 1
[1, 3, 4

In [19]:
T.df['response_eq_example'].value_counts()

False    100
Name: response_eq_example, dtype: int64

In [20]:
T.df['response_length_gt_steps'].value_counts()

False    100
Name: response_length_gt_steps, dtype: int64

In [21]:
T.df['response_eq_all_steps'].value_counts()

False    100
Name: response_eq_all_steps, dtype: int64

In [22]:
T.df['proportion_of_all_steps'].value_counts()

0.5    30
0.6    27
0.4    24
0.7    14
0.8     5
Name: proportion_of_all_steps, dtype: int64

In [23]:
T.df['meta_selections'].value_counts()

[1]          43
[1, 3, 5]    36
[1, 3]       18
[1, 5]        3
Name: meta_selections, dtype: int64

In [24]:
T.df['meta_selections_pc'].value_counts().sort_index()

0.333333    43
0.666667    21
1.000000    36
Name: meta_selections_pc, dtype: int64

In [25]:
T.df['object_selections'].value_counts()

[]        76
[4]       21
[2, 4]     2
[2]        1
Name: object_selections, dtype: int64

In [26]:
T.df['object_selections_pc'].value_counts().sort_index()

0.0    76
0.5    22
1.0     2
Name: object_selections_pc, dtype: int64

In [27]:
# templates = ['A1', 'B1', 'C1', 'D1']

# output = Path('outputs/2024-05-24T16:16:02.060434.json')
# output = Path('outputs/2024-05-24T16:16:02.061578.json')
# # output = Path('outputs/2024-05-24T16:16:02.062049.json')
# with output.open() as f:
#     data = json.load(f)


# eq_example = []
# length_gt_steps = []
# eq_steps = []
# for t in templates:
#     T = PromptParticipant(output, t)
#     # count occurences != none
#     eq_example.append(sum(T.df['response_eq_example']))
#     length_gt_steps.append(sum(T.df['response_length_gt_steps']))
#     eq_steps.append(sum(T.df['response_eq_all_steps']))

# # create dataframe from results
# comp_df = pd.DataFrame({
#     'template': templates,
#     'num_steps': STEP_NUM.values(),
#     'eq_example': eq_example,
#     'length_gt_steps': length_gt_steps,
#     'eq_steps': eq_steps
# })

# print(comp_df)